Question 1.a

In [2]:
P_hosteler = 0.60  # Probability that a student is a hosteler
P_day_scholar = 0.40  # Probability that a student is a day scholar
P_A_given_hosteler = 0.30  # Probability of scoring an A grade given that the student is a hosteler
P_A_given_day_scholar = 0.20  # Probability of scoring an A grade given that the student is a day scholar

# Calculate the probability using Bayes' Theorem
# P(H | A) = (P(A | H) * P(H)) / (P(A | H) * P(H) + P(A | D) * P(D))
P_hosteler_given_A = (P_A_given_hosteler * P_hosteler) / ((P_A_given_hosteler * P_hosteler) + (P_A_given_day_scholar * P_day_scholar))

# Display the result
print(f"The probability that the student is a hosteler given an A grade is: {P_hosteler_given_A:.4f}")

The probability that the student is a hosteler given an A grade is: 0.6923


1.b

In [3]:
P_Disease = 0.01
P_No_Disease = 1 - P_Disease
P_Positive_given_Disease = 0.99
P_Positive_given_No_Disease = 0.02

P_Disease_given_Positive = (P_Positive_given_Disease * P_Disease) / (
    P_Positive_given_Disease * P_Disease + P_Positive_given_No_Disease * P_No_Disease
)

print(f"The probability of having the disease given a positive test result is: {P_Disease_given_Positive:.4f}")


The probability of having the disease given a positive test result is: 0.3333


Naive Bayes Classifier

In [10]:
import csv

In [11]:
# Load the data from the CSV file
def load_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            data.append(row)
    return data

# Split the data into training and testing sets
def split_data(data, split_ratio):
    split_index = int(len(data) * split_ratio)
    return data[:split_index], data[split_index:]

# Calculate the probabilities for each class and feature
def calculate_probabilities(train_data):
    class_probs = {}
    feature_probs = {}

    total_samples = len(train_data)
    for row in train_data:
        label = row['Play']
        if label not in class_probs:
            class_probs[label] = 0
        class_probs[label] += 1

        for feature in row:
            if feature != 'Play':
                if (feature, row[feature], label) not in feature_probs:
                    feature_probs[(feature, row[feature], label)] = 0
                feature_probs[(feature, row[feature], label)] += 1

    for label in class_probs:
        class_probs[label] /= total_samples

    for feature_tuple in feature_probs:
        feature_probs[feature_tuple] /= class_probs[feature_tuple[2]]

    return class_probs, feature_probs

# Calculate the probability of a given class for a set of features
def calculate_class_probability(class_probs, feature_probs, features, label):
    probability = class_probs[label]
    for feature, value in features.items():
        if (feature, value, label) in feature_probs:
            probability *= feature_probs[(feature, value, label)]
    return probability

# Predict the class for a set of features
def predict(class_probs, feature_probs, features):
    best_label = None
    best_prob = -1

    for label in class_probs:
        probability = calculate_class_probability(class_probs, feature_probs, features, label)
        if probability > best_prob:
            best_prob = probability
            best_label = label

    return best_label

# Calculate accuracy, precision, and recall
def evaluate(test_data, class_probs, feature_probs):
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0

    for row in test_data:
        actual = row['Play']
        features = {feature: row[feature] for feature in row if feature != 'Play'}
        predicted = predict(class_probs, feature_probs, features)

        if actual == 'Yes' and predicted == 'Yes':
            true_positives += 1
        elif actual == 'No' and predicted == 'Yes':
            false_positives += 1
        elif actual == 'No' and predicted == 'No':
            true_negatives += 1
        elif actual == 'Yes' and predicted == 'No':
            false_negatives += 1

    accuracy = (true_positives + true_negatives) / len(test_data)
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)

    return accuracy, precision, recall



In [12]:
def main():
    file_path = 'data.csv'
    data = load_data(file_path)
    train_data, test_data = split_data(data, split_ratio=0.8)

    class_probs, feature_probs = calculate_probabilities(train_data)
    accuracy, precision, recall = evaluate(test_data, class_probs, feature_probs)

    print(f'Accuracy: {accuracy:.2f}')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')

    # Predict if the Player should play for sunny weather
    features_to_predict = {'Weather': 'Sunny'}
    prediction = predict(class_probs, feature_probs, features_to_predict)
    print(f'Prediction for Sunny Weather: {prediction}')


In [17]:
if __name__ == '__main__':
    main()

Accuracy: 0.67
Precision: 0.67
Recall: 1.00
Prediction for Sunny Weather: Yes


Additional Question

In [15]:
import pandas as pd

training_data = {
    'text document':['I love this sandwhich', 'this is an amazing place','I feel very good about these beers', 'this is my best work','what an awesom view', 'I do not like this restaurent','I am tired of this stuff', 'I cant deal with this','he is my sworn enemy', 'my boss is horible', 'this is an awesom place', 'I do not like the taste of this juice', 'I love to dance', 'I am sick and tired of this place','what a great holiday', 'this is a bad locality to stay', 'we will have good fun tomorrow', 'I went to my enemy\'s home today'],
    'label': ['pos','pos','pos','pos','pos','neg','neg','neg','neg','neg','pos','neg','pos','neg','pos','neg','pos','neg']
}

df = pd.DataFrame(training_data)
df.to_csv('centement.csv')

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the dataset from the CSV file
df = pd.read_csv('centement.csv')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text document'], df['label'], test_size=0.2, random_state=42)

# Create a CountVectorizer to convert text data into numerical features
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Create a Multinomial Naïve Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test_vectorized)

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='pos')
recall = recall_score(y_test, y_pred, pos_label='pos')

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
